In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from scipy.stats import uniform, expon, poisson, describe
import math

import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pickle

In [3]:
n = 500000
D = 2

ell = 0.8 * np.sqrt(2)

x_init = np.array([0, 0])
x_goal = np.array([ell, 0])

In [4]:
# r_n function
def r(n, D):
    return (n ** (-1/(2*D))) /2
    # designed for n = 500000

In [12]:
filename = 'slurm scripts/data/08-12-20/path_points/'

points = []
distances = []
lengths_based = []
counts = []

T_n_indices = defaultdict(list)

for i in range(1,1001):
    #if distances[i] == float('inf'):
    #    S_n.append(float('inf'))
    #    continue
    f = open(filename+str(i)+'-dim2-n500000_pathpoints.pkl', "rb")
    pathpoints = pickle.load(f)
    points.append(pathpoints)
    s_n = 0
    for j in range(len(pathpoints)-2):
        s_n += np.linalg.norm(pathpoints[j+1] - pathpoints[j])
    lengths_based.append(s_n)
    distances.append(s_n + np.linalg.norm(pathpoints[-1] - pathpoints[-2]))
    counts.append(len(pathpoints)-2)
    T_n_indices[len(pathpoints)-2].append(i)
    
    if pathpoints[-1][0] != 0.9 or pathpoints[-1][1] != 0.9:
        print(i)

        
distances = np.array(distances)
lengths_based = np.array(lengths_based)
counts_based = np.array(counts)

In [5]:
(r(n,D) ** 2) * math.pi * n

555.3603672697957

In [6]:
def run_simulation(seed, process=1):
    np.random.seed(seed) 
    
    results = defaultdict(lambda: float('inf'))
    results['seed'] = seed
    
    X = [x_init[0]]
    Y = [x_init[1]]
    
    lmbda = (n * (r(n, D) ** 2) * math.pi)

    R = [None]
    Theta = [None]
    S = [None]
    Gamma = [0]

    t = 0
    while np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) > r(n, D) and t < 1000:
        t += 1 # timestep index

        # three samples for each time step
        R.append(r(n, D) * (np.random.uniform() ** 0.5))
        S.append(np.random.choice(a=[-1, 1]))
        if process == 1:
            Theta.append((math.pi / lmbda) * np.random.exponential())   # uses asymptotics on the # of points in a ball explicitly
        elif process == 2:
            n_B = np.random.poisson((r(n, D) ** 2) * math.pi * n)       # simulates the number of points in a ball, leading to a mixture of exponentials
            Theta.append((math.pi / n_B) * np.random.exponential())
                                   
        # now we can determine the rest
        X.append(X[t-1] + R[t] * np.cos(Gamma[t-1] - Theta[t] * S[t]))
        Y.append(Y[t-1] + R[t] * np.sin(Gamma[t-1] - Theta[t] * S[t]))

        g = np.arcsin((R[t] / np.linalg.norm(np.array([X[t], Y[t]]) - x_goal)) * np.sin(Theta[t] * S[t]))
        Gamma.append(Gamma[t-1] + g)
        
    if t < 1000:
        results['T'] = t
        results['length'] = sum(R[1:])
        results['last_point'] = (X[-1], Y[-1])
        results['distance_to_goal'] = np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) 
        results['R'] = R
        results['Theta'] = Theta
        results['S'] = S
        results['X'] = X
        results['Y'] = Y
        
    return results

In [7]:
simulation_outputs_1 = defaultdict(dict)
simulation_outputs_2 = defaultdict(dict)

for i in range(1, 1001):
    simulation_outputs_1[i] = run_simulation(i * 100, 1)
    simulation_outputs_2[i] = run_simulation(i * 200, 2)

In [9]:
def show_results(idx, key, process):
    if process==1:
        print(simulation_outputs_1[idx][key])
    else:
        print(simulation_outputs_2[idx][key])

interact(show_results, idx=widgets.IntSlider(min=1, max=1000, step=1, value=1), key=widgets.Dropdown(options=simulation_outputs_1[1].keys()), process=widgets.Dropdown(options=[1,2]))

interactive(children=(IntSlider(value=1, description='idx', max=1000, min=1), Dropdown(description='key', opti…

<function __main__.show_results(idx, key, process)>

In [10]:
lengths_free = {1 : [], 2 : []}
for k in simulation_outputs_1.keys():
    lengths_free[1].append(simulation_outputs_1[k]['length'])
    lengths_free[2].append(simulation_outputs_2[k]['length'])
    
lengths_free[1] = np.array(lengths_free[1])
lengths_free[2] = np.array(lengths_free[2])

counts_free = {1 : [], 2 : []}
for k in simulation_outputs_1.keys():
    counts_free[1].append(simulation_outputs_1[k]['T'])
    counts_free[2].append(simulation_outputs_2[k]['T'])
    
counts_free[1] = np.array(counts_free[1])
counts_free[2] = np.array(counts_free[2])

In [13]:
# statistics on length

print(describe(lengths_based))
print(describe(lengths_free[1]))
print(describe(lengths_free[2]))

DescribeResult(nobs=1000, minmax=(1.1126248979058297, 1.1311135195351911), mean=1.1196871228929786, variance=2.1605416879607834e-05, skewness=0.3758413955784031, kurtosis=-0.8310809528509653)
DescribeResult(nobs=1000, minmax=(1.1126039115026416, 1.1310496314200864), mean=1.1196094678529052, variance=2.1384207965097682e-05, skewness=0.37987571247289986, kurtosis=-0.8313209236787373)
DescribeResult(nobs=1000, minmax=(1.1126048347744781, 1.1308823497845906), mean=1.1198069537601623, variance=2.0717493869762104e-05, skewness=0.2798078600849421, kurtosis=-0.89861498923351)


In [14]:
# statistics on T

print(describe(counts_based)) # graph based
print(describe(counts_free[1]))  # graph free process 1
print(describe(counts_free[2]))  # graph free process 2

DescribeResult(nobs=1000, minmax=(77, 92), mean=84.914, variance=5.680284284284284, skewness=-0.014300944202705558, kurtosis=0.00994488869075516)
DescribeResult(nobs=1000, minmax=(80, 100), mean=89.467, variance=11.654565565565562, skewness=0.28261040610904153, kurtosis=-0.054461693721175486)
DescribeResult(nobs=1000, minmax=(79, 101), mean=89.356, variance=10.143407407407407, skewness=0.1815592025796908, kurtosis=0.24819666824362097)


In [15]:
# statistics on T

print(describe((counts_based ** 0.5) * lengths_based )) # graph based
print(describe((counts_free[1] ** 0.5) * lengths_free[1] ))  # graph free process 1
print(describe((counts_free[2] ** 0.5) * lengths_free[2] ))  # graph free process 2

DescribeResult(nobs=1000, minmax=(9.806764663307925, 10.758327615888245), mean=10.316789313941499, variance=0.0232903912891977, skewness=-0.10933426973532663, kurtosis=-0.060126368980016665)
DescribeResult(nobs=1000, minmax=(10.016728963728564, 11.279316451981247), mean=10.58821380921983, variance=0.04431402856297593, skewness=0.23810589685434153, kurtosis=-0.017154192372287547)
DescribeResult(nobs=1000, minmax=(9.961840806083927, 11.298070376922942), mean=10.583745663062233, variance=0.03897455788749313, skewness=0.08842521891947569, kurtosis=0.10166361829699255)


In [26]:
# need to run one more simulation, where both lambda_n and T_n are big enough (lambda_n > 500, T_n > 250)

# r_n function
def test(n, D):
    return (n ** (-1/(2*D))) / 5 

n_test = 5* 10 **6

print(test(n_test, 2) ** 2 * math.pi * n_test)
print(ell / test(n_test, 2))

280.9925892416291
267.49612199056884


In [32]:
array = []

for i in range(5 * 10 ** 6):
    if True:
        point = np.random.uniform(size=2)
        if (point[0]-0.5)**2 - 1.997 * (point[0]-0.5) * (point[1]-0.5) + (point[1]-0.5)**2 > 1/1800:
            continue
        array.append(point)
    else:
        pass 
    
print(len(array))

159180
